In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from pbpstats.client import Client
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/"

league = "NBA"
year = 2023
season_yr = str(year)
season_type = "Regular Season"
games_id = pbp_season(league=league,season_yr=season_yr,season_type=season_type)
games_list = pbp_games(games_id)

In [ ]:
player_dict = get_players(league=league,from_year=year, to_year=year)
player_id = [player['pID'] for player in player_dict]
team_dict = get_teams(league=league) # Creating Team Dictionary
team_id = [x['TeamID'] for x in team_dict ]

In [ ]:
game = games_list[0]
possession = game.possessions.items[2]
possession_event = possession.events
event = possession_event[0]

In [ ]:
# Get Location Data
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
game_id, period, clock, seconds_remaining,poss_length = [],[],[],[],[]
team_id, player1_id= [],[]
locX, locY, distance, shot_value, shot_type = [],[],[],[],[]
is_made, is_putback, is_assisted, player2_id  = [],[],[],[]
len_diffp = 0
pos_store = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, FieldGoal):
                try: 
                    game_id.append(possession_event.game_id)
                    period.append(possession_event.period)
                    clock.append(possession_event.clock)
                    seconds_remaining.append(possession_event.seconds_remaining)
                    poss_length.append(possession_event.seconds_since_previous_event)
                    team_id.append(possession_event.team_id)
                    player1_id.append(possession_event.player1_id)
                    locX.append(possession_event.locX)
                    locY.append(possession_event.locY)
                    distance.append(possession_event.distance)
                    shot_value.append(possession_event.shot_value)
                    shot_type.append(possession_event.shot_type)
                    is_made.append(possession_event.is_made)
                    
                    is_assisted.append(possession_event.is_assisted)
                    if possession_event.is_assisted:
                        player2_id.append(possession_event.player2_id)
                    else:
                        player2_id.append(0)
                    ii += 1
                    # print("ii = {0}".format(ii))
                    is_putback.append(possession_event.is_putback)
                except:
                    is_putback.append(False)
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
player1_name = []
for pID in player1_id:
    player1_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player1_name = list(itertools.chain(*player1_name))
player2_name = []
for pID in player2_id:
    if pID==0:
        player2_name.append(['None'])
    else:     
        player2_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player2_name = list(itertools.chain(*player2_name))
team_name = []
for tID in team_id:  
    team_name.append([team['Team'] for team in team_dict if team['TeamID'] == tID])
team_name = list(itertools.chain(*team_name))

In [ ]:
len(team_name)

In [ ]:
data = pd.DataFrame({'game_id':game_id,'period':period,'clock':clock,
                    'seconds_remaining':seconds_remaining,'poss_length':poss_length,
                    'team_id':team_id,'team_name':team_name,'player_id':player1_id,
                    'player_name':player1_name,'locX': locX,'locY':locY, 'distance':distance,
                    'shot_value':shot_value,'shot_type':shot_type,'is_made':is_made,
                    'is_assisted':is_assisted,'player_ast_id':player2_id,'player_ast_name':player2_name,
                    'is_putback':is_putback})
# data = data.sort_values(by=['Assists'],ascending=False)
# data = data.reset_index(drop=True)
data.head()

In [ ]:
data.to_csv(f'ShotLocationData\{league}_Shot_Loc_{year}.csv',index=False)